# Setup and Configuration

This notebook sets up the configuration and initializes AWS/SageMaker sessions for the Two-Stage Recommendation System.


In [ ]:
import boto3
import sagemaker
from sagemaker.feature_store.feature_store import FeatureStore

# Configuration
def get_config():
    """Get configuration dictionary."""
    return {
        'region': "ap-south-1",
        'role_arn': "arn:aws:iam::487512486150:role/recommendationsystem-sagemaker-role",
        'bucket': "amazon-sagemaker-local-dev-store",
        'user_feature_group_name': "all-beauty-users-1766218384",  # Update with actual name
        'item_feature_group_name': "all-beauty-items-1766218384"   # Update with actual name
    }

def initialize_sessions(config: dict = None):
    """Initialize AWS and SageMaker sessions."""
    if config is None:
        config = get_config()
    
    # Initialize Boto3 session
    boto_session = boto3.Session(region_name=config['region'])
    
    # Create SageMaker session
    sagemaker_session = sagemaker.Session(
        boto_session=boto_session,
        default_bucket=config['bucket']
    )
    
    # Feature Store clients
    featurestore_runtime = boto_session.client(
        service_name='sagemaker-featurestore-runtime',
        region_name=config['region']
    )
    
    sagemaker_client = boto_session.client(
        service_name='sagemaker',
        region_name=config['region']
    )
    
    feature_store = FeatureStore(sagemaker_session=sagemaker_session)
    
    print(f"Initialized SageMaker session in {config['region']}")
    print(f"Default bucket: {config['bucket']}")
    
    return {
        'boto_session': boto_session,
        'sagemaker_session': sagemaker_session,
        'featurestore_runtime': featurestore_runtime,
        'sagemaker_client': sagemaker_client,
        'feature_store': feature_store,
        'config': config
    }


In [ ]:
# Get configuration
config = get_config()

# Update Feature Group names with your actual names
# You can find them by listing feature groups (see cell below)
config['user_feature_group_name'] = "all-beauty-users-1766218384"  # Update this
config['item_feature_group_name'] = "all-beauty-items-1766218384"   # Update this

print("Configuration:")
for key, value in config.items():
    print(f"  {key}: {value}")


In [ ]:
# Initialize sessions
sessions = initialize_sessions(config)

# Extract sessions for use in other notebooks
boto_session = sessions['boto_session']
sagemaker_session = sessions['sagemaker_session']
featurestore_runtime = sessions['featurestore_runtime']
sagemaker_client = sessions['sagemaker_client']
feature_store = sessions['feature_store']


## List Feature Groups

Run this cell to find your Feature Group names:


In [ ]:
# List all feature groups in your region
response = sagemaker_client.list_feature_groups()

print("Available Feature Groups:")
for fg in response['FeatureGroupSummaries']:
    print(f"- {fg['FeatureGroupName']} (Created: {fg['CreationTime']})")
